In [1]:
# train7에서 만든 모델 testset 만들기


In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from keras import metrics

import tensorflow_addons.metrics

C:\Users\SJ\anaconda3\envs\study\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\SJ\anaconda3\envs\study\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want

In [3]:
import warnings

# 경고 무시
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
train = train.drop(['user_id'], axis=1)
test_id = test['user_id'].values
test = test.drop(['user_id'], axis=1)



In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  object 
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [7]:
# preferred_difficulty_level 열 인코딩
train.loc[train['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
train.loc[train['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
train.loc[train['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2

test.loc[test['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
test.loc[test['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
test.loc[test['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2
train['preferred_difficulty_level'] = train['preferred_difficulty_level'].astype(int)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [9]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 데이터 스케일링을 위한 StandardScaler 인스턴스 생성
scaler = StandardScaler()
categoric_col =['subscription_type']
label_encoders = {}
for col in categoric_col:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].fit_transform(test[col])

In [10]:
# 'payment_pattern' 열에 대한 고유값 확인
unique_values = train['payment_pattern'].unique()

# 각 고유값에 대해 새로운 열 생성 및 0 또는 1로 설정
for value in unique_values:
    train[f'payment_pattern_{value}'] = (train['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
train.drop('payment_pattern', axis=1, inplace=True)

for value in unique_values:
    test[f'payment_pattern_{value}'] = (test['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
test.drop('payment_pattern', axis=1, inplace=True)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  int32  
 11  customer_inquiry_history           10000 non-null  int6

In [12]:
columns_to_scale = train.columns.drop('target')  # 실제 열 이름으로 대체해야 함

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# 선택한 열을 Min-Max Scaling
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
test[columns_to_scale] = scaler.fit_transform(test[columns_to_scale])

In [13]:
train_x = train.drop('target', axis = 1)
train_y = train['target']
test_x = test

In [14]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [15]:
train_x = X_train
train_y = y_train

In [16]:
ros = RandomOverSampler(random_state=42)
train_x, train_y = ros.fit_resample(train_x, train_y)

In [26]:
train_x

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern_5,payment_pattern_6,payment_pattern_7,payment_pattern_0,payment_pattern_1,payment_pattern_3,payment_pattern_4,payment_pattern_2
0,0.454545,0.071429,0.525012,0.070676,0.347826,0.538462,0.330938,0.333333,1.00,0.0,0.0,0.1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.590909,0.642857,0.601930,0.369298,0.478261,0.500000,0.660292,0.083333,1.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.545455,0.928571,0.447911,0.003759,0.565217,0.538462,0.373111,0.250000,0.75,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.818182,0.178571,0.424160,0.177374,0.260870,0.346154,0.525984,0.083333,0.75,0.0,1.0,0.1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.863636,0.071429,0.644118,0.152848,0.565217,0.307692,0.602216,0.333333,0.75,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,0.272727,0.357143,0.390083,0.344516,0.695652,0.192308,0.492663,0.416667,1.00,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9910,0.090909,1.000000,0.597827,0.113368,0.217391,0.346154,0.153263,0.500000,0.50,0.0,0.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9911,0.136364,0.035714,0.655861,0.047916,0.869565,0.538462,0.598357,0.500000,1.00,0.0,1.0,0.4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9912,0.272727,0.428571,0.306884,0.083876,0.739130,0.538462,0.577087,0.166667,0.75,0.0,1.0,0.3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
train_y

0       1
1       1
2       1
3       1
4       1
       ..
9909    0
9910    0
9911    0
9912    0
9913    0
Name: target, Length: 9914, dtype: int64

In [19]:
train_x.fillna(0, inplace=True)
test_x.fillna(0, inplace=True)

In [20]:
import tensorflow as tf

def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))

    return tf.sqrt(tf.reduce_mean(squared_error))

def loss_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

def metric_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

def f1_macro(y_true, y_pred):
    def precision_m(y_true, y_pred):
        true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        predicted_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
        return precision

    def recall_m(y_true, y_pred):
        true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
        return recall

    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)

    f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    return f1

def binary_cross_entropy(y_true, y_pred):
    return tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)

# def f1_binary(y_true, y_pred):
#     def precision_m(y_true, y_pred):
#         true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
#         return precision
# 
#     def recall_m(y_true, y_pred):
#         true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
#         possible_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
#         return recall
# 
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
# 
#     f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
#     return f1

def f1_macro(y_true, y_pred):
    def recall_m(y_true, y_pred):
        true_positives = tf.math.count_nonzero(tf.math.round(y_true * y_pred), axis=0)
        possible_positives = tf.math.count_nonzero(tf.math.round(y_true), axis=0)
        recall = tf.math.divide_no_nan(true_positives, possible_positives)
        return recall

    def precision_m(y_true, y_pred):
        true_positives = tf.math.count_nonzero(tf.math.round(y_true * y_pred), axis=0)
        predicted_positives = tf.math.count_nonzero(tf.math.round(y_pred), axis=0)
        precision = tf.math.divide_no_nan(true_positives, predicted_positives)
        return precision

    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)

    f1 = tf.math.divide_no_nan(2 * (precision * recall), (precision + recall))
    f1_macro = tf.reduce_mean(f1)

    return f1_macro


In [21]:
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.85, patience=5, min_lr=0.000001),
    tf.keras.callbacks.TerminateOnNaN()
]

In [22]:
def create_model():

    input_layer = tf.keras.Input(shape=(len(train_x.columns), ))

    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    # x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    # x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    # x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)



    # output_layer = tf.keras.layers.Dense(1)(x)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=binary_cross_entropy,
                  metrics=['accuracy',f1_macro]
                  )

    return model

model = create_model()
history = model.fit(train_x.astype('float32'), train_y.astype('float32'),
                    epochs=400,
                    callbacks=callbacks_list,
                    validation_split=0.2)

Epoch 1/400
248/248 [==============================] - 2s 3ms/step - loss: 0.6636 - accuracy: 0.6192 - f1_macro: 0.7606 - val_loss: 0.8584 - val_accuracy: 0.0232 - val_f1_macro: 0.0291 - lr: 0.0010
Epoch 2/400
248/248 [==============================] - 0s 2ms/step - loss: 0.6548 - accuracy: 0.6191 - f1_macro: 0.7615 - val_loss: 0.9055 - val_accuracy: 0.0232 - val_f1_macro: 0.0291 - lr: 0.0010
Epoch 3/400
248/248 [==============================] - 0s 2ms/step - loss: 0.6525 - accuracy: 0.6193 - f1_macro: 0.7615 - val_loss: 0.8416 - val_accuracy: 0.0287 - val_f1_macro: 0.0291 - lr: 0.0010
Epoch 4/400
248/248 [==============================] - 0s 2ms/step - loss: 0.6482 - accuracy: 0.6193 - f1_macro: 0.7616 - val_loss: 0.8700 - val_accuracy: 0.0363 - val_f1_macro: 0.0291 - lr: 0.0010
Epoch 5/400
248/248 [==============================] - 0s 1ms/step - loss: 0.6477 - accuracy: 0.6197 - f1_macro: 0.7591 - val_loss: 0.8908 - val_accuracy: 0.0287 - val_f1_macro: 0.0291 - lr: 0.0010
Epoch 6/40

In [23]:
 # 모델을 사용하여 X_test 데이터로 예측
y_pred = model.predict(X_test.astype('float32'))

# 예측값을 이진으로 변환 (예: 임계값 0.5 사용)
y_pred_binary = (y_pred > 0.5).astype(int)

# F1_macro 계산
from sklearn.metrics import f1_score

f1_macro = f1_score(y_test, y_pred_binary, average='macro')

# 결과 출력
print(f'F1_macro 값: {f1_macro}')

63/63 [==============================] - 0s 1ms/step
F1_macro 값: 0.49862336840546345


In [33]:
# 3405
sample_submission = pd.read_csv('../data/sample_submission.csv')
predictions = model.predict(test_x.astype('float32'))
binary_predictions = (predictions >= 0.5).astype(int)
sample_submission["target"] = binary_predictions
sample_submission.to_csv("../submission_data/12test.csv", index=False)

313/313 [==============================] - 0s 777us/step


In [398]:
# 5181
def create_model():

    input_layer = tf.keras.Input(shape=(len(train_x.columns), ))

    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    # x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    # x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    # x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)



    # output_layer = tf.keras.layers.Dense(1)(x)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=binary_cross_entropy,
                  metrics=['accuracy',f1_macro]
                  )

    return model

model = create_model()
history = model.fit(train_x.astype('float32'), train_y.astype('float32'),
                    epochs=400,
                    callbacks=callbacks_list,
                    validation_split=0.2)

Epoch 1/300
248/248 [==============================] - 3s 3ms/step - loss: 0.7026 - accuracy: 0.5669 - f1_macro: 0.6705 - val_loss: 0.8886 - val_accuracy: 0.0923 - val_f1_macro: 0.0290 - lr: 0.0010
Epoch 2/300
248/248 [==============================] - 0s 2ms/step - loss: 0.6597 - accuracy: 0.6023 - f1_macro: 0.7276 - val_loss: 0.8929 - val_accuracy: 0.1599 - val_f1_macro: 0.0267 - lr: 0.0010
Epoch 3/300
248/248 [==============================] - 0s 2ms/step - loss: 0.6503 - accuracy: 0.6190 - f1_macro: 0.7418 - val_loss: 0.8813 - val_accuracy: 0.2063 - val_f1_macro: 0.0295 - lr: 0.0010
Epoch 4/300
248/248 [==============================] - 0s 2ms/step - loss: 0.6428 - accuracy: 0.6304 - f1_macro: 0.7488 - val_loss: 0.8517 - val_accuracy: 0.2809 - val_f1_macro: 0.0299 - lr: 0.0010
Epoch 5/300
248/248 [==============================] - 0s 2ms/step - loss: 0.6391 - accuracy: 0.6365 - f1_macro: 0.7504 - val_loss: 0.8732 - val_accuracy: 0.2259 - val_f1_macro: 0.0292 - lr: 0.0010
Epoch 6/30

# Catboost

In [291]:
submit.to_csv("../submission_data/5.csv", index = False)

In [46]:
pred_final = model_xgb.predict(test)

In [47]:
submit = pd.DataFrame({
    'user_id': test_id,
    'target': pred_final.flatten()
})

In [48]:
submit.to_csv("../submission_data/5.csv", index = False)

In [30]:
import tensorflow as tf
a = []
f1_macro = 0

while f1_macro < 0.5203:
    def rmsle(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
    
        squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))
    
        return tf.sqrt(tf.reduce_mean(squared_error))
    
    def loss_fn(y_true, y_pred):
        return rmsle(y_true, y_pred)
    
    def metric_fn(y_true, y_pred):
        return rmsle(y_true, y_pred)
    
    def f1_macro(y_true, y_pred):
        def precision_m(y_true, y_pred):
            true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
            predicted_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_pred, 0, 1)))
            precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
            return precision
    
        def recall_m(y_true, y_pred):
            true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
            possible_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true, 0, 1)))
            recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
            return recall
    
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
    
        f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
        return f1
    
    def binary_cross_entropy(y_true, y_pred):
        return tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    
    # def f1_binary(y_true, y_pred):
    #     def precision_m(y_true, y_pred):
    #         true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    #         predicted_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_pred, 0, 1)))
    #         precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    #         return precision
    # 
    #     def recall_m(y_true, y_pred):
    #         true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    #         possible_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true, 0, 1)))
    #         recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    #         return recall
    # 
    #     precision = precision_m(y_true, y_pred)
    #     recall = recall_m(y_true, y_pred)
    # 
    #     f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    #     return f1
    
    def f1_macro(y_true, y_pred):
        def recall_m(y_true, y_pred):
            true_positives = tf.math.count_nonzero(tf.math.round(y_true * y_pred), axis=0)
            possible_positives = tf.math.count_nonzero(tf.math.round(y_true), axis=0)
            recall = tf.math.divide_no_nan(true_positives, possible_positives)
            return recall
    
        def precision_m(y_true, y_pred):
            true_positives = tf.math.count_nonzero(tf.math.round(y_true * y_pred), axis=0)
            predicted_positives = tf.math.count_nonzero(tf.math.round(y_pred), axis=0)
            precision = tf.math.divide_no_nan(true_positives, predicted_positives)
            return precision
    
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
    
        f1 = tf.math.divide_no_nan(2 * (precision * recall), (precision + recall))
        f1_macro = tf.reduce_mean(f1)
    
        return f1_macro
    
    callbacks_list = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.85, patience=5, min_lr=0.000001),
        tf.keras.callbacks.TerminateOnNaN()
    ]
    
    # 5181
    def create_model():
    
        input_layer = tf.keras.Input(shape=(len(train_x.columns), ))
    
        x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
        x = tf.keras.layers.Dense(32, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
        x = tf.keras.layers.Dense(64, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
        x = tf.keras.layers.Dense(128, activation='relu')(x)
    
    
    
        # output_layer = tf.keras.layers.Dense(1)(x)
        output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
        model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
        model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                      loss=binary_cross_entropy,
                      metrics=['accuracy',f1_macro]
                      )
    
        return model
    
    model = create_model()
    history = model.fit(train_x.astype('float32'), train_y.astype('float32'),
                        epochs=400,
                        callbacks=callbacks_list,
                        validation_split=0.2)
    
    
    # 모델을 사용하여 X_test 데이터로 예측
    y_pred = model.predict(X_test.astype('float32'))
    
    # 예측값을 이진으로 변환 (예: 임계값 0.5 사용)
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    # F1_macro 계산
    from sklearn.metrics import f1_score
    
    f1_macro = f1_score(y_test, y_pred_binary, average='macro')
    a.append(f1_macro)
    # 결과 출력
    print(f'F1_macro 값: {f1_macro}')



Epoch 1/400
248/248 [==============================] - 2s 2ms/step - loss: 0.6802 - accuracy: 0.5900 - f1_macro: 0.7189 - val_loss: 0.8163 - val_accuracy: 0.1261 - val_f1_macro: 0.0285 - lr: 0.0010
Epoch 2/400
248/248 [==============================] - 0s 2ms/step - loss: 0.6614 - accuracy: 0.6086 - f1_macro: 0.7334 - val_loss: 0.9686 - val_accuracy: 0.0872 - val_f1_macro: 0.0290 - lr: 0.0010
Epoch 3/400
248/248 [==============================] - 0s 2ms/step - loss: 0.6486 - accuracy: 0.6143 - f1_macro: 0.7386 - val_loss: 0.8489 - val_accuracy: 0.2431 - val_f1_macro: 0.0288 - lr: 0.0010
Epoch 4/400
248/248 [==============================] - 0s 2ms/step - loss: 0.6440 - accuracy: 0.6201 - f1_macro: 0.7418 - val_loss: 0.9006 - val_accuracy: 0.1760 - val_f1_macro: 0.0296 - lr: 0.0010
Epoch 5/400
248/248 [==============================] - 0s 2ms/step - loss: 0.6416 - accuracy: 0.6279 - f1_macro: 0.7442 - val_loss: 0.9425 - val_accuracy: 0.1533 - val_f1_macro: 0.0281 - lr: 0.0010
Epoch 6/40

KeyboardInterrupt: 

In [32]:
a

[0.4978563689683941,
 0.493843032306589,
 0.5080547830193629,
 0.5030682267708916,
 0.5102412062059436]